In [1]:
!pip install tweepy
!pip install streamlit
!pip install pyngrok
!pip install pillow
!pip install requests
!pip install yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.1 MB/s eta 0:00:00


In [3]:
%%writefile app.py

import yt_dlp
import streamlit as st
import tweepy
import requests
from PIL import Image
from io import BytesIO
import os

# Twitter API credentials
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAABdOxAEAAAAAxRrl9G9Xb25GQWeKl9mZgXa%2FrzA%3Dx01dtjJMkIAXNEip38OZue2VBmQOIwg46ImXe14mBfFpuJlzFd"

# Initialize Tweepy Client
client = tweepy.Client(bearer_token=BEARER_TOKEN)

def fetch_image_content(image_url):
    """Fetch and process an image from a URL."""
    try:
        response = requests.get(image_url, timeout=10)
        response.raise_for_status()
        return Image.open(BytesIO(response.content))
    except Exception as e:
        st.write(f"Error fetching image: {e}")
        return None

def download_video(video_url, save_path):
    """Download a video from the given URL and save it locally."""
    ydl_opts = {
        "quiet": True,
        "format": "best[ext=mp4]",
        "outtmpl": save_path,
        "noplaylist": True,
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        return save_path
    except Exception as e:
        st.write(f"Error downloading video: {e}")
        return None

def get_latest_tweets_with_media(username, max_items=10):
    """Fetch latest tweets with text, associated images, and videos."""
    user = client.get_user(username=username)
    if not user.data:
        return []

    user_id = user.data.id

    response = client.get_users_tweets(
        id=user_id,
        tweet_fields=["attachments"],
        expansions=["attachments.media_keys"],
        media_fields=["url", "type", "variants"],
        exclude=["retweets", "replies"],
        max_results=max_items
    )

    tweet_data = []

    if response.data:
        for tweet in response.data:
            text = tweet.text

            images = []
            videos = []
            if hasattr(tweet, "attachments") and tweet.attachments is not None:
                if "media_keys" in tweet.attachments:
                    for media_key in tweet.attachments["media_keys"]:
                        media = next(
                            (media for media in response.includes.get("media", []) if media["media_key"] == media_key), None
                        )
                        if media:
                            if media.type == "photo":
                                images.append(media.url)
                            elif media.type == "video":
                                if "variants" in media:
                                    video_url = max(
                                        media.variants,
                                        key=lambda v: v.get("bitrate", 0) if v.get("content_type") == "video/mp4" else 0
                                    ).get("url", "")
                                    if video_url:
                                        videos.append(video_url)

            tweet_data.append({"text": text, "images": images, "videos": videos})

    return tweet_data

# Streamlit App
st.title("Twitter Profile Data Extractor")

username = st.text_input("Enter Twitter username (without @):")
if username:
    st.write(f"Fetching data for @{username}...")
    tweets_with_media = get_latest_tweets_with_media(username, max_items=10)

    if tweets_with_media:
        for i, tweet in enumerate(tweets_with_media, start=1):
            st.subheader(f"Tweet {i}:")
            st.write(tweet["text"])

            # Display images
            for img_url in tweet["images"]:
                image = fetch_image_content(img_url)
                if image:
                    st.image(image, caption=f"Image from Tweet {i}", use_column_width=True)

            # Display videos
            for vid_url in tweet["videos"]:
                st.write(f"Downloading and displaying video for Tweet {i}...")
                video_path = f"video_{i}.mp4"
                downloaded_path = download_video(vid_url, video_path)
                if downloaded_path:
                    st.video(downloaded_path)
                    os.remove(downloaded_path)  # Clean up after displaying
                else:
                    st.warning("Could not download or display video.")
    else:
        st.warning("No tweets, images, or videos found!")


Writing app.py


In [4]:
# Import ngrok
from pyngrok import ngrok

# Set your authtoken
ngrok.set_auth_token("2ohUKqk37HcGbvwN0s8Y1E2WNxE_39z1gVF3bYq9vFSEm7Wzq") # Replace YOUR_AUTHTOKEN with your actual authtoken

# Kill any existing ngrok processes
ngrok.kill()

# Start Streamlit with nohup
!nohup streamlit run app.py &

# Create a public URL with ngrok to access the app
public_url = ngrok.connect(addr='8501')
print(f"Public URL: {public_url}")

nohup: appending output to 'nohup.out'
Public URL: NgrokTunnel: "https://64c3-34-106-79-17.ngrok-free.app" -> "http://localhost:8501"


In [5]:
ngrok.kill()